# Data Pipeline

## Part 1: Webscraping news articles

### Install packages

In [4]:
import html5lib
import bs4
import requests
from bs4 import BeautifulSoup

### Access contents

In [6]:
headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9"}
URLS = ["https://www.vox.com/donald-trump/401174/trump-fired-generals", "https://www.foxnews.com/world/trump-says-minerals-deal-has-been-pretty-much-negotiated-zelenskyy-meeting-slated-friday"]
for url in URLS:
    r = requests.get(url, headers=headers)
    print(r.content)

b'<!DOCTYPE html><html lang="en-US"><meta name="bingbot" content="nocache"/><head><meta charSet="utf-8"/><meta name="twitter:card" content="summary_large_image"/><meta name="twitter:site" content="@voxdotcom"/><meta property="fb:app_id" content="549923288395304"/><meta property="og:site_name" content="Vox"/><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"/><meta name="apple-mobile-web-app-title" content="Vox"/><link rel="alternate" type="application/rss+xml" title="Vox" href="/rss/index.xml"/><title>Trump, Hegseth purge key military leaders. Here\xe2\x80\x99s what\xe2\x80\x99s at stake. | Vox</title><meta name="robots" content="index,follow,max-image-preview:large"/><meta name="description" content="How the Friday night massacre could change America\xe2\x80\x99s armed forces."/><meta property="og:title" content="What Trump\xe2\x80\x99s military purge was really about"/><meta property="og:description" content="How the Friday night massacre could chan

### Parse contents

In [8]:
soup = BeautifulSoup(r.content, 'html5lib')
print(soup.prettify())

<!DOCTYPE html>
<html data-n-head="%7B%22lang%22:%7B%22ssr%22:%22en%22%7D%7D" data-n-head-ssr="" lang="en">
 <head>
  <title>
   Volodymyr Zelenskyy, Donald Trump come to agreement on minerals deal | Fox News
  </title>
  <meta content="IE=edge,chrome=1" data-n-head="ssr" http-equiv="X-UA-Compatible"/>
  <meta content="text/html; charset=utf-8" data-hid="content-type" data-n-head="ssr" http-equiv="content-type"/>
  <meta content="on" data-hid="x-dns-prefetch-control" data-n-head="ssr" http-equiv="x-dns-prefetch-control"/>
  <meta charset="utf-8" data-n-head="ssr"/>
  <meta content="width=device-width, minimum-scale=1.0, initial-scale=1.0" data-hid="viewport" data-n-head="ssr" name="viewport"/>
  <meta content="//static.foxnews.com/static/orion/styles/img/fox-news/favicons/mstile-70x70.png" data-n-head="ssr" name="msapplication-square70x70logo"/>
  <meta content="//static.foxnews.com/static/orion/styles/img/fox-news/favicons/mstile-150x150.png" data-n-head="ssr" name="msapplication-squa

In [9]:
soup.find_all("script")

[<script data-n-head="ssr" type="text/javascript">
           (function(h,o,u,n,d) {
             h=h[d]=h[d]||{q:[],onReady:function(c){h.q.push(c)}}
             d=o.createElement(u);d.async=1;d.src=n
             n=o.getElementsByTagName(u)[0];n.parentNode.insertBefore(d,n)
           })(window,document,'script','https://www.datadoghq-browser-agent.com/datadog-rum-v4.js','DD_RUM')
           DD_RUM.onReady(function() {
             DD_RUM.init({
               clientToken: 'pubc5794d5f3bcb446a2c16a37a20423236',
               applicationId: '5932e398-c53f-4964-bbbd-fea737be8819',
               site: 'datadoghq.com',
               service: 'www.foxnews.com',
               env: 'prod',
               // Specify a version number to identify the deployed version of your application in Datadog
               version: '1.0.0',
               sampleRate: 1,
               sessionReplaySampleRate: 0,
               trackInteractions: true,
               trackResources: true,
           

In [10]:
soup.find('data-n-head'=="ssr")

<html data-n-head="%7B%22lang%22:%7B%22ssr%22:%22en%22%7D%7D" data-n-head-ssr="" lang="en"><head>
    <title>Volodymyr Zelenskyy, Donald Trump come to agreement on minerals deal | Fox News</title><meta content="IE=edge,chrome=1" data-n-head="ssr" http-equiv="X-UA-Compatible"/><meta content="text/html; charset=utf-8" data-hid="content-type" data-n-head="ssr" http-equiv="content-type"/><meta content="on" data-hid="x-dns-prefetch-control" data-n-head="ssr" http-equiv="x-dns-prefetch-control"/><meta charset="utf-8" data-n-head="ssr"/><meta content="width=device-width, minimum-scale=1.0, initial-scale=1.0" data-hid="viewport" data-n-head="ssr" name="viewport"/><meta content="//static.foxnews.com/static/orion/styles/img/fox-news/favicons/mstile-70x70.png" data-n-head="ssr" name="msapplication-square70x70logo"/><meta content="//static.foxnews.com/static/orion/styles/img/fox-news/favicons/mstile-150x150.png" data-n-head="ssr" name="msapplication-square150x150logo"/><meta content="//static.foxn

In [11]:
headline_text=[]
table = soup.find('script')
for row in table.find_all('script', attrs = {'data-n-head':'ssr', 'type': 'application/ld+json'}):
    return(row)
    #headline = row.find('headline').text.strip()
    #text = row.find('articleBody').text.strip()
    #title_text.append({'headline': headline, 'text':text})

SyntaxError: 'return' outside function (171558838.py, line 4)

In [ ]:
headline_text

## Part 2: Uploading and parsing subjective intensifiers data

#### Import dependencies

In [19]:
import numpy as np
import pandas as pd

#### Read in data

In [21]:
#https://stackoverflow.com/questions/64966365/parse-sgml-using-beautiful-soup
with open("acl-99-data/subjectivity.data", "r") as f:
    entries = f.read()
    soup = BeautifulSoup(entries, "html.parser")
print(soup)

<txt art="FEAT" docnum="891101-0108" tnum="??">
<mc cluster="2" cmt="rs" soa="other" uid="1">
<wf pos="DT" quote="0" stem="an">An</wf>
<mc cluster="2" cmt="" soa="JANADDEDTHISLINE" uid="2">
<wf pos="CC" quote="0" stem="but">but</wf>
<s s_id="891101-0108_0_1" snum="1">
<mc cluster="2" cmt="rs" soa="other" uid="3">
<wf nc="pronoun" pos="PRP" quote="0" stem="it">It</wf>
<s s_id="891101-0108_0_2" snum="2">
<mc cluster="2" soa="other" uid="4">
<wf nc="noun" pos="NNP" quote="0" stem="amateur">Amateur</wf>
<p pnum="1">
<s s_id="891101-0108_1_0" snum="0">
<mc cluster="1" soa="other" uid="5">
<wf nc="pronoun" pos="PRP" quote="0" stem="he">He</wf>
<mc cluster="2" cmt="" soa="JANADDEDTHISLINE" uid="6">
<wf pos="DT" quote="0" stem="an">an</wf>
<mc cluster="2" cmt="" soa="JANADDEDTHISLINE" uid="7">
<wf pos="CC" quote="0" stem="and">and</wf>
<s s_id="891101-0108_1_1" snum="1">
<mc1 cluster="2" soa="se.ms" uid="8">
<wf nc="pronoun" pos="PRP" quote="0" stem="he">He</wf>
<mc1 cluster="2" soa="se.ms" ui

#### Basic assessment of data

In [31]:
entries=[]
table = soup.find('txt')
m_c = table.find_all_next('mc')
#print(m_c[0])
#entries_dict = {}
print(m_c[0].mc)

<mc cluster="2" cmt="" soa="JANADDEDTHISLINE" uid="2">
<wf pos="CC" quote="0" stem="but">but</wf>
<s s_id="891101-0108_0_1" snum="1">
<mc cluster="2" cmt="rs" soa="other" uid="3">
<wf nc="pronoun" pos="PRP" quote="0" stem="it">It</wf>
<s s_id="891101-0108_0_2" snum="2">
<mc cluster="2" soa="other" uid="4">
<wf nc="noun" pos="NNP" quote="0" stem="amateur">Amateur</wf>
<p pnum="1">
<s s_id="891101-0108_1_0" snum="0">
<mc cluster="1" soa="other" uid="5">
<wf nc="pronoun" pos="PRP" quote="0" stem="he">He</wf>
<mc cluster="2" cmt="" soa="JANADDEDTHISLINE" uid="6">
<wf pos="DT" quote="0" stem="an">an</wf>
<mc cluster="2" cmt="" soa="JANADDEDTHISLINE" uid="7">
<wf pos="CC" quote="0" stem="and">and</wf>
<s s_id="891101-0108_1_1" snum="1">
<mc1 cluster="2" soa="se.ms" uid="8">
<wf nc="pronoun" pos="PRP" quote="0" stem="he">He</wf>
<mc1 cluster="2" soa="se.ms" uid="9">
<wf pos="CC" quote="0" stem="and">and</wf>
<s s_id="891101-0108_1_2" snum="2">
<mc cluster="2" cmt="rs" soa="other" uid="10">
<w

In [27]:
len(m_c)

812

In [ ]:
..
if 'cluster' in row.mc:
        entries_dict['subj'] = row.mc['cluster']
    if 'cmt' in row.mc:
        entries_dict['cmt'] = row.mc['cmt']
    if 'soa' in row.mc:
        entries_dict['soa'] = row.mc['soa']
    if 'uid' in row.mc:
        entries_dict['uid'] = row.mc['uid']
    if 'nc' in row.wf:
        entries_dict['nc'] = row.wf['nc']
    if 'pos' in row.wf:
        entries_dict['pos'] = row.wf['pos']
    if 'quote' in row.wf:
        entries_dict['quote'] = row.wf['quote']
    if 'stem' in row.wf:
        entries_dict['stem'] = row.wf['stem']
    if 'pnum' in row.p:
        entries_dict['pnum'] = row.p['pnum']
    if 's_id' in row.s:
        entries_dict['s_id'] = row.s['s_id']
    if 'snum' in row.s:
        entries_dict['snum'] = row.s['snum']
    entries.append(entries_dict)